In [ ]:
# Global packages
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_curve, roc_auc_score, auc
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
from torchvision import models, transforms
from torch.utils.data import DataLoader, Subset
from torch.utils.data.sampler import SubsetRandomSampler

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Loading in the Dataset

In [ ]:
normalize = transforms.Normalize(
    mean=[0.3322, 0.0275, 0.1132],
    std=[0.2215, 0.0965, 0.3152],
)

transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize,
])

# Load the dataset
dataset = datasets.ImageFolder("./Dataset_BUSI/Dataset_BUSI_with_GT/train/", transform=transform)

# Get the labels from the dataset
labels = np.array(dataset.targets)

# Split the dataset into train and test sets while maintaining class proportions
train_indices, test_indices = train_test_split(np.arange(len(dataset)), test_size=0.2, random_state=42, stratify=labels)

# Create Subset datasets for train and test
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [ ]:
# Making sure train and test indices don't overlap
assert len(set(train_indices).intersection(test_indices)) == 0, "Indices overlap between train and validation sets."

# Checking the distribution
This is to make sure that the distributions are equal across the Original, Train, and Test datasets before training and evaluation.

In [ ]:
def print_distribution(header, distribution):
    """
    Prints the class distribution of a dataset.

    Args:
        header (str): Header text to indicate the dataset type.
        distribution (dict): A dictionary containing class labels and their ratios.

    Returns:
        None
    """
    print("=" * 40)
    print(f"{header} Dataset Ratio:")
    for label, ratio in distribution.items():
        class_name = "Benign" if label == 0 else "Malignant"
        print(f"{class_name}: {ratio:.2%} of the {header.lower()} dataset")

# Count the occurrences of each label in the dataset
label_counts = Counter(dataset.targets)
total_samples = len(dataset)
labels = np.array(dataset.targets)

# Filter indices for the train and test sets
train_indices = [idx for idx in range(len(dataset)) if idx in train_indices]
test_indices = [idx for idx in range(len(dataset)) if idx in test_indices]

def calculate_distribution(indices):
    """
    Calculates the class distribution for a given set of indices.

    Args:
        indices (list): List of indices for a specific dataset subset.

    Returns:
        dict: A dictionary containing class labels and their ratios in the subset.
    """
    total_samples = len(indices)
    return {class_label: np.sum(labels[indices] == class_label) / total_samples for class_label in np.unique(labels)}

# Calculate class distribution for the train and test sets
train_class_distribution = calculate_distribution(train_indices)
test_class_distribution = calculate_distribution(test_indices)

# Print the class distribution of the main dataset
print("Main Dataset Ratio:")
for label, count in label_counts.items():
    label_str = dataset.classes[label]
    ratio = count / total_samples
    print(f"{label_str}: {count} samples ({ratio:.2%} of the main dataset)")

# Print class distribution for train and test sets
print_distribution("Train", train_class_distribution)
print_distribution("Test", test_class_distribution)


# Creating Optimizer
Currently the optimizer is an **Adam Optimizer**

In [ ]:
def create_optimizer(model,  # Function that creates an optimizer for a given model and hyperparameters
                    learning_rate_pretrained=0.00039710,  # Learning rate for pretrained layers
                    learning_rate_new=0.00083336,  # Learning rate for new layers
                    weight_decay=0.0073691,  # Weight decay (L2 regularization) coefficient
                    beta1=0.9,  # Exponential decay rate for the first moment estimates (Adam parameter)
                    beta2=0.999,  # Exponential decay rate for the second moment estimates (Adam parameter)
                    eps=1e-8,  # Small constant to prevent division by zero (Adam parameter)
                    amsgrad=False):  # Flag for using the AMSGrad variant of Adam optimizer

    # Separate model parameters into two groups: pretrained and new
    params_pretrained = []  # Parameters from the pretrained layers
    params_new = []         # Parameters from the new layers

    for name, param in model.named_parameters():
        if 'classifier.6' in name:
            params_new.append(param)  # Parameters from the new layers
        else:
            params_pretrained.append(param)  # Parameters from the pretrained layers

    '''Setting Loss Functions and Optimizers'''
    # Create an Adam optimizer with different learning rates for the two parameter groups
    optimizer = optim.Adam([
        {'params': params_pretrained, 'lr': learning_rate_pretrained},  # Pretrained layer parameters with specific learning rate
        {'params': params_new, 'lr': learning_rate_new},                # New layer parameters with different learning rate
    ], weight_decay=weight_decay, betas=(beta1, beta2), eps=eps, amsgrad=amsgrad)

    return optimizer

# Train Function
Training function for the VGG16 model.

In [ ]:
def training(model, device, train_loader, criterion, optimizer):
    """
    Training function for a neural network model.

    Args:
        model (nn.Module): The neural network model to be trained.
        device (torch.device): The device (e.g., 'cuda' or 'cpu') to use for training.
        train_loader (DataLoader): DataLoader containing training data.
        criterion: The loss function.
        optimizer: The optimization algorithm.

    Returns:
        float: The average training loss for one epoch.
    """

    # Move the model to the specified device
    model.to(device)
    # Set the model in training mode
    model.train()

    # Initialize variables to keep track of training loss and total samples
    train_loss, total_samples = 0.0, 0

    # Loop through the batches in the training data
    for images, labels in train_loader:
        # Move data to the specified device
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients of the optimizer
        optimizer.zero_grad()

        # Forward pass through the model
        outputs = model(images)
        # Calculate the loss
        loss = criterion(outputs, labels.unsqueeze(1).float())

        # Backpropagation: Compute gradients
        loss.backward()
        # Update the model's parameters
        optimizer.step()

        # Accumulate the training loss for the current batch
        train_loss += loss.item() * images.size(0)
        # Count the total number of processed samples
        total_samples += labels.size(0)

    # Calculate the average training loss for the entire epoch
    avg_train_loss = train_loss / total_samples

    return avg_train_loss

# Testing Function (Evaluation)

Evaluation function for the VGG16 model.

In [ ]:
def testing(model, device, val_loader, criterion):
    """
    Evaluation function for a neural network model.

    Args:
        model (nn.Module): The neural network model to be evaluated.
        device (torch.device): The device (e.g., 'cuda' or 'cpu') to use for evaluation.
        val_loader (DataLoader): DataLoader containing validation data.
        criterion: The loss function.

    Returns:
        float: The average validation loss.
    """

    # Move the model to the specified device
    model.to(device)
    # Set the model in evaluation mode
    model.eval()

    # Initialize variables to keep track of validation loss and total samples
    val_loss, total_samples = 0.0, 0

    # Disable gradient calculation during evaluation
    with torch.no_grad():
        # Loop through the batches in the validation data
        for images, labels in val_loader:
            # Move data to the specified device
            images, labels = images.to(device), labels.to(device)
            # Forward pass through the model
            outputs = model(images)
            # Calculate the loss
            loss = criterion(outputs, labels.unsqueeze(1).float())

            # Accumulate the validation loss for the current batch
            val_loss += loss.item() * labels.size(0)
            # Count the total number of processed samples
            total_samples += labels.size(0)

    # Calculate the average validation loss
    avg_val_loss = val_loss / total_samples

    return avg_val_loss

# Creating Train and Validation Loaders
Creates data loaders for training and validation using given indices.

In [ ]:
def create_train_val_loader(train_dataset, train_idx, val_idx, batch_size):
    """
    Creates data loaders for training and validation using given indices.

    Args:
        train_dataset (Dataset): The training dataset.
        train_idx (list): List of indices for the training set.
        val_idx (list): List of indices for the validation set.
        batch_size (int): Batch size for the data loaders.

    Returns:
        train_loader (DataLoader): DataLoader for the training data.
        val_loader (DataLoader): DataLoader for the validation data.
    """

    # Create subset samplers for training and validation indices
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    # Check for overlapping indices between train and validation sets
    assert len(set(train_idx).intersection(val_idx)) == 0, "Indices overlap between train and validation sets."

    # Define data loaders for training and validation data in this fold
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_subsampler,
        shuffle=True  # Shuffle the training data for each epoch
    )

    val_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=val_subsampler
    )

    return train_loader, val_loader


# Setting Parameters

In [ ]:
'''Setting training parameters'''
num_frozen_layers = 5
num_epochs = 20 # epochs
batch_size = 32 # batch size
k=5 # fold number

'''Setting cross-validation parameters'''
fold_results={'train_loss': [],
              'val_loss': []} # dictionary to memorize all the scores

skfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42) # sklearn library

'''Setting Loss Function'''
criterion = nn.BCEWithLogitsLoss() # Binary Cross Entropy Loss with Sigmoid Function (only have to classify 2 things -> 0 or 1): With multiple classes you generally want to use softmax()

# Checking distributions of each fold
Validates the class distribution and indices of a cross-validation fold.

In [ ]:
def check_distribution(k, fold, train_indices, train_idx, val_idx):
    """
    Validates the class distribution and indices of a cross-validation fold.

    Args:
        k (int): Total number of folds in cross-validation.
        fold (int): Current fold number.
        train_indices (list): List of indices for the entire dataset.
        train_idx (list): List of indices for the training set in the current fold.
        val_idx (list): List of indices for the validation set in the current fold.

    Returns:
        None
    """

    # Calculate class distribution for training and validation sets
    train_class_distribution = {class_label: np.sum(labels[train_indices][train_idx] == class_label) for class_label in np.unique(labels)}
    val_class_distribution = {class_label: np.sum(labels[train_indices][val_idx] == class_label) for class_label in np.unique(labels)}

    # Calculate class ratios for training and validation sets
    total_train_samples = len(train_idx)
    total_val_samples = len(val_idx)
    train_class_ratios = {class_label: count / total_train_samples for class_label, count in train_class_distribution.items()}
    val_class_ratios = {class_label: count / total_val_samples for class_label, count in val_class_distribution.items()}

    # Check dataset size and indices consistency
    total_samples = len(train_indices)
    train_samples = len(train_idx)
    val_samples = len(val_idx)
    assert total_samples == train_samples + val_samples, "Train and validation set sizes do not add up."

    # Check for index uniqueness and non-overlapping
    assert len(set(train_idx).intersection(val_idx)) == 0, "Indices overlap between train and validation sets."

    print("=" * 40)
    print(f"Fold {fold + 1}/{k}")

    # Print class ratios for training set
    for train_label, train_ratio in train_class_ratios.items():
        class_name = "Benign" if train_label == 0 else "Malignant"
        print(f"{class_name}: {train_ratio:.2%} of the CV-TRAIN dataset")

    # Print class ratios for validation set
    for val_label, val_ratio in val_class_ratios.items():
        class_name = "Benign" if val_label == 0 else "Malignant"
        print(f"{class_name}: {val_ratio:.2%} of the CV-VALID dataset")

    print("=" * 40)

# Running the Model

In [ ]:
model = None
best_models_per_fold = []

for fold, (train_idx, val_idx) in enumerate(skfold.split(train_dataset, labels[train_indices])):
    # Check distribution of indices
    check_distribution(k, fold, train_indices, train_idx, val_idx)

    best_val_loss = float('inf')

    train_loader, val_loader = create_train_val_loader(train_dataset, train_idx, val_idx, batch_size)

    # Initiate the VGG16
    model = models.vgg16(pretrained=True)

    in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features=4096, out_features=1)

    # Freezing some layers
    for layer in model.features[:num_frozen_layers]:
        for param in layer.parameters():
            param.requires_grad = False

    # Initiating our optimizer
    optimizer = create_optimizer(model)

    # Iterative epochs
    for epoch in range(num_epochs):
        # Train and Val
        avg_train_loss = training(model, device, train_loader, criterion, optimizer)
        avg_val_loss = testing(model, device, val_loader, criterion)

        print("Epoch:{}/{} AVG Training Loss:{:.6f} AVG Val Loss:{:.6f}".format(epoch + 1, num_epochs, avg_train_loss, avg_val_loss))

        # Update best validation loss and save model weights if necessary
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()

        # Save results
        fold_results['train_loss'].append(avg_train_loss)
        fold_results['val_loss'].append(avg_val_loss)

    model.load_state_dict(best_model_state)

    best_models_per_fold.append((model, best_val_loss))


# Testing the model
Testing the model on the holdout evaluation set that we created at the beginning.

In [ ]:
# Create a subset sampler for the test indices
test_subsampler = torch.utils.data.SubsetRandomSampler(test_indices)

# Create a DataLoader for the test data
test_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=test_subsampler,
    shuffle=False
)

# List of threshold values for evaluation
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# List to store F1 scores for each fold
f1_scores_per_fold = []

# Extract models from the list of best models per fold
extracted_models = [model for model, _ in best_models_per_fold]

# Loop through extracted models for evaluation
for model in extracted_models:
    model.eval()
    all_labels = []
    all_predicted_labels_per_threshold = {threshold: [] for threshold in thresholds}

    # Disable gradient calculation during evaluation
    with torch.no_grad():
        # Loop through test data batches
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # Apply sigmoid activation to model outputs
            predicted_probs = torch.sigmoid(outputs)
            
            # Iterate through thresholds and make predictions
            for threshold in all_predicted_labels_per_threshold:
                predicted_labels = (predicted_probs >= threshold).float()  # Apply threshold
                all_predicted_labels_per_threshold[threshold].extend(predicted_labels.cpu().numpy())

            all_labels.extend(labels.cpu().numpy())

    f1_scores = []

    # Calculate F1 scores for different thresholds
    for threshold, predicted_labels in all_predicted_labels_per_threshold.items():
        f1_scores.append(f1_score(all_labels, predicted_labels, average='micro'))

    f1_scores_per_fold.append(f1_scores)

# Find the highest F1 score and corresponding model and threshold
max_f1 = -1
best_model_idx = -1
best_threshold_idx = -1

for model_idx, model in enumerate(extracted_models):
    for threshold_idx, f1_score_value in enumerate(f1_scores_per_fold[model_idx]):
        if f1_score_value > max_f1:
            max_f1 = f1_score_value
            best_model_idx = model_idx
            best_threshold_idx = threshold_idx

# Get the best model and corresponding threshold
best_model = extracted_models[best_model_idx]
best_threshold = thresholds[best_threshold_idx]
best_predicted_labels = all_predicted_labels_per_threshold[best_threshold]

# Calculate precision and recall at the best point
best_precision = precision_score(all_labels, best_predicted_labels, average='micro')
best_recall = recall_score(all_labels, best_predicted_labels, average='micro')

# Performance Measurements

In [ ]:
# Find the index of the tuple with the lowest validation loss in the list
index_of_lowest_loss = min(range(len(best_models_per_fold)), key=lambda i: best_models_per_fold[i][1])

# Retrieve the best model and its associated lowest validation loss using the found index
best_model_with_lowest_loss, lowest_loss = best_models_per_fold[index_of_lowest_loss]

print("Index of the tuple with the lowest best_val_loss:", index_of_lowest_loss)
print("Lowest loss:", lowest_loss)

In [ ]:
# Print the evaluation results
print(f"Highest F1 Score: {max_f1}")
print(f"Best Model Index: {best_model_idx}")
print(f"Best Threshold: {best_threshold}")
print(f"Precision Score at Best Point: {best_precision}")
print(f"Recall Score at Best Point: {best_recall}")

In [ ]:
# Graphing
fold_idx = num_epochs * best_model_idx
best_train_loss = fold_results['train_loss'][fold_idx : fold_idx+num_epochs]
best_test_loss = fold_results['val_loss'][fold_idx : fold_idx+num_epochs]

plt.figure()
plt.plot(range(1, num_epochs + 1), best_train_loss, label='Training Loss')
plt.plot(range(1, num_epochs + 1), best_test_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Calculate the average F1 scores across different folds
avg_f1_scores = np.mean(f1_scores_per_fold, axis=0)

# Find the index of the best threshold that maximizes the average F1 score
best_threshold_avg_index = np.argmax(avg_f1_scores)

# Retrieve the actual threshold value corresponding to the best average F1 score
best_threshold_avg = thresholds[best_threshold_avg_index]

print("Avg Threshold:", best_threshold_avg)
print("Avg F1 Score:", np.max(avg_f1_scores))


In [ ]:
# Calculate the average training and test loss at specific epochs
avg_train_loss = []
avg_test_loss = []
for epoch in range(num_epochs):
    avg_train_loss_at_epoch = np.mean(np.array(fold_results['train_loss'])[epoch::num_epochs], axis=0)
    avg_test_loss_at_epoch = np.mean(np.array(fold_results['val_loss'])[epoch::num_epochs], axis=0)
    avg_train_loss.append(avg_train_loss_at_epoch)
    avg_test_loss.append(avg_test_loss_at_epoch)

plt.figure()
plt.plot(range(1, num_epochs + 1), avg_train_loss, label='Training Loss')
plt.plot(range(1, num_epochs + 1), avg_test_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()